In [36]:
import time, threading, queue, sys, gc
from typing import Tuple

import numpy as np
import pyqtgraph as pg
from PyQt6 import QtCore
from collections import deque

import logging, logging.handlers


In [37]:
# PULSE_WIDTH
# 1rps, 512 slots, 1/512 ~ 0.002
# 0.5 rps, 512 slots, 1/256 ~ 0.004

# ISSUE
# 0.2 * 2048 = 409.6
# 0.2 * 2048 * 0.5 = 204.8
# 0.5 * 2048 * 0.5 = 512 <--
# 0.125 * 2048 = 256

In [ ]:
# ------------------------------------------------------------  parameters
DEBUG = True  # set True for jitter log
SAMPLE_RATE = 100_000  # Hz
#CHUNK_SEC = 0.125  # s
GEN_CHUNK_SEC = 0.02           # daq rate
PROC_INTERVAL = 0.125          # process rate

CHUNK_SEC     = GEN_CHUNK_SEC
N_SAMPLES_GEN = int(SAMPLE_RATE * GEN_CHUNK_SEC)
SAMPLES_PROC = int(PROC_INTERVAL * SAMPLE_RATE) # 0.125s * 100kHz = 12500
QUEUE_DEPTH = 40  # raw AB backlog
QUAD_DEPTH = 40  # processed backlog (same)
RUN_SEC = 600  # auto‑stop after 10 s
DISPLAY_SEC = RUN_SEC+600
PLOT_SEC = 0.02  # *** fixed x-axis window width (s) ***
GUI_INTERVAL_MS = 50  # update interval (ms) e.g. 50ms = 20Hz

PULSE_HEIGHT = 5.0  # amplitude
INPUT_VELOCITY = 0.5  # rps
PULSE_WIDTH = 1 / (INPUT_VELOCITY * 512)  # period  (s)
PULSE_DUTY = 0.5  # duty
PULSE_PHASE_A = 0.0  # phase offset (s)
PULSE_PHASE_B = -PULSE_WIDTH / 4  # phase offset (s)

QUADPULSE_WIDTH = 0.00025  # width (s) assuming 4x given 1 rps
THRESHOLD_DEFAULT = 2.5  # logic threshold (V)


# ------------------------------------------------------------  queues & stop flag
buf_q = queue.Queue(maxsize=QUEUE_DEPTH)  # raw (t, A, B)
quad_q = queue.Queue(maxsize=QUAD_DEPTH)  # processed (t, A, B, quad)
stop_writer = threading.Event()

log_q = queue.Queue(maxsize=0)
queue_h = logging.handlers.QueueHandler(log_q)
logger = logging.getLogger("debug")
logger.setLevel(logging.INFO)
logger.addHandler(queue_h)



In [ ]:
def gen_chunk_pulse(
    t: np.ndarray,
    *,
    height: float = PULSE_HEIGHT,
    width: float = PULSE_WIDTH,
    duty: float = PULSE_DUTY,
    phase: float = 0.0,
) -> np.ndarray:
    mod = (t + phase) % width
    return np.where(mod < duty * width, height, 0.0).astype(np.float32)


REL_AXIS_GEN = (
    np.arange(N_SAMPLES_GEN, dtype=np.float32) / SAMPLE_RATE
)  # 0 ... 0.2 s

# ------------------------------------------------------------  AB → direction → quad helpers


def gen_pulse_direction(
    dA: np.ndarray,
    dB: np.ndarray,
    *,
    threshold: float,
    prev_A: bool | None = None,
    prev_B: bool | None = None,
) -> tuple[np.ndarray, bool, bool]:
    """
    Detect direction for one block **with perfect edge coverage**.

    Parameters
    ----------
    dA, dB : ndarray[float32]
        Analog levels of phase-A / phase-B for the current block.
    threshold : float
        Logic threshold [V].
    prev_A, prev_B : bool | None
        Logical state of A/B *at the end of the PREVIOUS block*.
        • If None (first block), the function falls back to the
          “self-shift” method used before.

    Returns
    -------
    dir_log : ndarray[int8]
        +1 = CW edge, –1 = CCW edge, 0 = no edge.
    last_A, last_B : bool
        Logical state of A/B at the *end* of this block — feed these
        into the next call to avoid losing the boundary edge.
    """
    # --- current logic level ----------------------------------------
    A = dA > threshold
    B = dB > threshold

    # --- previous sample for XOR -----------------------------------
    if prev_A is None:  # first block → old behaviour
        A_prev = np.concatenate(([A[0]], A[:-1]))
        B_prev = np.concatenate(([B[0]], B[:-1]))
    else:               # use states carried over from last block
        A_prev = np.concatenate(([prev_A], A[:-1]))
        B_prev = np.concatenate(([prev_B], B[:-1]))

    dir_log = (B_prev ^ A).astype(int) - (A_prev ^ B).astype(int)

    return dir_log.astype(np.int8), bool(A[-1]), bool(B[-1])

def pulse_count(dir_log: np.ndarray) -> int:
    return int(np.sum(dir_log))


def gen_quad_pulse(
    t: np.ndarray, dir_log: np.ndarray, width: float, height: float, sampling_rate: int
) -> np.ndarray:
    samples = int(width * sampling_rate)
    if samples <= 0:
        return np.zeros_like(dir_log, dtype=np.float32)
    base = np.full(samples, height, dtype=np.float32)
    return np.convolve(dir_log, base, mode="full")[: len(t)]


# ------------------------------------------------------------  producer thread
def generator() -> None:
    """Generate AB rectangular-wave chunks at real-time cadence."""
    chunk_idx = 0
    next_t = time.perf_counter()
    while not stop_writer.is_set():
        base = chunk_idx * GEN_CHUNK_SEC
        t_axis = REL_AXIS_GEN + base
        pulse_A = gen_chunk_pulse(t_axis, phase=PULSE_PHASE_A)
        pulse_B = gen_chunk_pulse(t_axis, phase=PULSE_PHASE_B)
        try:
            buf_q.put_nowait((t_axis, pulse_A, pulse_B))
        except queue.Full:
            pass

        chunk_idx += 1
        next_t += GEN_CHUNK_SEC
        sleep = next_t - time.perf_counter()
        if sleep > 0:
            time.sleep(sleep)
        else:
            next_t = time.perf_counter()


# ------------------------------------------------------------  consumer thread
def log_listener():
    handler = logging.StreamHandler(sys.stdout)
    listener = logging.handlers.QueueListener(log_q, handler)
    listener.start()
    stop_writer.wait()
    listener.stop()


def process_chunk(
    t: np.ndarray, dA: np.ndarray, dB: np.ndarray
) -> Tuple[int, np.ndarray]:
    dir_log = gen_pulse_direction(dA, dB, threshold=THRESHOLD_DEFAULT)
    return pulse_count(dir_log), gen_quad_pulse(
        t, dir_log, QUADPULSE_WIDTH, PULSE_HEIGHT, SAMPLE_RATE
    )

'''
def consumer() -> None:
    last_ts = time.perf_counter()
    cum_count = 0
    while not stop_writer.is_set():
        try:
            t, pulse_A, pulse_B = buf_q.get(timeout=0.5)
        except queue.Empty:
            continue
        now = time.perf_counter()
        count, quad_sig = process_chunk(t, pulse_A, pulse_B)
        cum_count += count
        velocity = count / CHUNK_SEC / 2048  # counts per second

        if DEBUG:
            jitter = (now - last_ts) * 1e3
            # print(
            #    f"EPOCH = {now:.2f} jitter = {jitter:6.2f} ms  Δc={count:+d}  c={cum_count}  v={velocity:.7f}"
            # )
            logger.info(
                "EPOCH = %f jitter = %6.2f ms  Δc=%+d  c=%d  v=%6.3f",
                now,
                jitter,
                count,
                cum_count,
                velocity,
            )
        last_ts = now

        try:
            quad_q.put_nowait(
                (t, pulse_A, pulse_B, quad_sig, t[-1], cum_count, velocity)
            )
        except queue.Full:
            pass

        del t, pulse_A, pulse_B, quad_sig
        # gc.collect() # uncomment to force garbage collection
        buf_q.task_done()'''


def processor() -> None:
    ring_t  = deque()   # ring buffer for t
    ring_a  = deque()   # ring buffer for A
    ring_b  = deque()   # ring buffer for B
    last_A = last_B = None

    next_proc = time.perf_counter()
    cum_count = 0
    last_ts   = next_proc
    while not stop_writer.is_set():
        # ---------- データ取り込み (ノンブロッキング) ----------
        try:
            while True:
                t, pA, pB = buf_q.get_nowait()
                ring_t.extend(t); ring_a.extend(pA); ring_b.extend(pB)
                buf_q.task_done()
        except queue.Empty:
            pass
        # ---------- データ取り込み (ブロッキング) ----------
        now = time.perf_counter()
        if now < next_proc:
            time.sleep(next_proc - now)
            continue
        next_proc += PROC_INTERVAL          # 次の処理時間をセット

        if len(ring_t) < SAMPLES_PROC:
            continue                        # サンプル不足ならスキップ

        # retriev samples from ring buffers
        t_blk  = np.array([ring_t.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)
        a_blk  = np.array([ring_a.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)
        b_blk  = np.array([ring_b.popleft() for _ in range(SAMPLES_PROC)], dtype=np.float32)

        dir_log, last_A, last_B  = gen_pulse_direction(a_blk, b_blk, threshold=THRESHOLD_DEFAULT,prev_A=last_A, prev_B=last_B) # edge completion
        quad_sig = gen_quad_pulse(t_blk, dir_log, QUADPULSE_WIDTH, PULSE_HEIGHT, SAMPLE_RATE)
        delta_cnt = pulse_count(dir_log)
        cum_count += delta_cnt
        velocity   = delta_cnt / PROC_INTERVAL / 2048

        if DEBUG:
            now = time.perf_counter()
            jitter = (now - last_ts) * 1e3
            # print(
            #    f"EPOCH = {now:.2f} jitter = {jitter:6.2f} ms  Δc={count:+d}  c={cum_count}  v={velocity:.7f}"
            # )
            logger.info(
                "EPOCH = %f, wall = %6.2f ms, jitter = %6.2f ms  Δc=%+d  c=%d  v=%6.3f, len(dir_log)=%d",
                now,
                jitter,
                (now - last_ts) * 1e3,
                delta_cnt,
                cum_count,
                velocity,
                len(dir_log),
            )
        last_ts = now

        try:
            quad_q.put_nowait((t_blk, a_blk, b_blk, quad_sig, t_blk[-1], cum_count, velocity))
        except queue.Full:
            pass

        del t_blk, a_blk, b_blk, quad_sig
        # gc.collect() # uncomment to force garbage collection


# ------------------------------------------------------------  GUI
HISTORY = int(SAMPLE_RATE * PLOT_SEC)
COUNT_HISTORY = int(RUN_SEC / CHUNK_SEC) * 6
VELO_HISTORY = COUNT_HISTORY


def start_gui() -> None:
    pg.setConfigOptions(useOpenGL=True, background="w", foreground="k")
    app = pg.mkQApp("Live plots")

    win = pg.GraphicsLayoutWidget(title="DEMO")
    win.resize(800, 600)
    win.show()

    # [0, 0] A/B -------------------------------------------------------
    plt_ab = win.addPlot(row=0, col=0, title="RAW A / B")
    curve_A = plt_ab.plot(pen=pg.mkPen("#ff4b00", width=3))
    curve_B = plt_ab.plot(pen=pg.mkPen("#005aff", width=3))
    plt_ab.setLabel(axis="left", text="Amplitude [V]")
    plt_ab.setLabel(axis="bottom", text="Time [s]")
    plt_ab.setYRange(-0.5, PULSE_HEIGHT + 0.5)

    # [1,0] Quad waveform --------------------------------------------
    plt_q = win.addPlot(row=1, col=0, title="Quad pulse")
    curve_Q = plt_q.plot(pen=pg.mkPen("m", width=3))
    plt_q.setLabel(axis="left", text="Amplitude [V]")
    plt_q.setLabel(axis="bottom", text="Time [s]")
    plt_q.setYRange(-0.5 - PULSE_HEIGHT, PULSE_HEIGHT + 0.5)

    # [0,1] count (fixed x‑axis) -------------------------------------------
    plt_cnt = win.addPlot(row=0, col=1, title="Quad count")
    curve_cnt = plt_cnt.plot(pen=pg.mkPen("#03af7a", width=3))
    plt_cnt.setXRange(0, RUN_SEC, padding=0)
    plt_cnt.enableAutoRange("x", False)
    plt_cnt.setLabel("left", "Count")
    plt_cnt.setLabel("bottom", "Time [s]")

    # [1,1] velocity (fixed x‑axis) ----------------------------------------
    plt_vel = win.addPlot(row=1, col=1, title="Velocity")
    curve_vel = plt_vel.plot(pen=pg.mkPen("#00a0e9", width=3))
    plt_vel.setXRange(0, RUN_SEC, padding=0)
    plt_vel.enableAutoRange("x", False)
    plt_vel.setLabel("left", "Velocity [c/s]")
    plt_vel.setLabel("bottom", "Time [s]")

    # buffers ---------------------------------------------------------
    xs = ya = yb = yq = np.empty(0, dtype=np.float32)
    xs_cnt = y_cnt = np.empty(0, dtype=np.float32)
    xs_vel = y_vel = np.empty(0, dtype=np.float32)

    def refresh():
        nonlocal xs, ya, yb, yq, xs_cnt, y_cnt, xs_vel, y_vel
        try:
            while True:
                t_ax, pA, pB, qsig, t_end, cum_cnt, vel = quad_q.get_nowait()
                xs = np.concatenate((xs, t_ax))[-HISTORY:]
                ya = np.concatenate((ya, pA))[-HISTORY:]
                yb = np.concatenate((yb, pB))[-HISTORY:]
                yq = np.concatenate((yq, qsig))[-HISTORY:]

                xs_cnt = np.append(xs_cnt, t_end)[-COUNT_HISTORY:]
                y_cnt = np.append(y_cnt, cum_cnt)[-COUNT_HISTORY:]

                xs_vel = np.append(xs_vel, t_end)[-VELO_HISTORY:]
                y_vel = np.append(y_vel, vel)[-VELO_HISTORY:]
                quad_q.task_done()
        except queue.Empty:
            pass

        # scrolling window for waveforms only
        if xs.size:
            start = xs[-1] - PLOT_SEC
            plt_ab.setXRange(start, xs[-1], padding=0)
            plt_q.setXRange(start, xs[-1], padding=0)

        # --- push data to curves ---
        curve_A.setData(xs, ya)
        curve_B.setData(xs, yb)
        curve_Q.setData(xs, yq)
        curve_cnt.setData(xs_cnt, y_cnt)
        curve_vel.setData(xs_vel, y_vel)

    timer = QtCore.QTimer()
    timer.timeout.connect(refresh)
    timer.start(GUI_INTERVAL_MS)

    # auto‑stop after RUN_SEC
    QtCore.QTimer.singleShot(DISPLAY_SEC * 1000, lambda: (stop_writer.set(), app.quit()))
    app.exec()

In [40]:
threading.Thread(target=log_listener, daemon=True).start() # start log listener
gen_th = threading.Thread(target=generator, daemon=True)
#con_th = threading.Thread(target=consumer, daemon=True)
proc_th = threading.Thread(target=processor, daemon=True)
gen_th.start(); proc_th.start()
#con_th.start()

start_gui()

# join threads and exit
stop_writer.set()
gen_th.join()
#con_th.join()
proc_th.join()

print("Graceful shutdown.")

EPOCH = 227030.273845, wall = 411.58 ms, jitter = 411.58 ms  Δc=+127  c=127  v= 0.496, len(dir_log)=12500
EPOCH = 227030.408090, wall = 134.24 ms, jitter = 134.24 ms  Δc=+128  c=255  v= 0.500, len(dir_log)=12500
EPOCH = 227030.517874, wall = 109.78 ms, jitter = 109.78 ms  Δc=+128  c=383  v= 0.500, len(dir_log)=12500
EPOCH = 227030.746436, wall = 228.56 ms, jitter = 228.56 ms  Δc=+128  c=511  v= 0.500, len(dir_log)=12500
EPOCH = 227030.875814, wall = 129.38 ms, jitter = 129.38 ms  Δc=+128  c=639  v= 0.500, len(dir_log)=12500
EPOCH = 227031.005243, wall = 129.43 ms, jitter = 129.43 ms  Δc=+128  c=767  v= 0.500, len(dir_log)=12500
EPOCH = 227031.120696, wall = 115.45 ms, jitter = 115.45 ms  Δc=+128  c=895  v= 0.500, len(dir_log)=12500
EPOCH = 227031.265585, wall = 144.89 ms, jitter = 144.89 ms  Δc=+128  c=1023  v= 0.500, len(dir_log)=12500
EPOCH = 227031.386680, wall = 121.09 ms, jitter = 121.09 ms  Δc=+128  c=1151  v= 0.500, len(dir_log)=12500
EPOCH = 227031.506936, wall = 120.26 ms, jit